                                                                                                 Vidhi Shah   Saurabh Bajoria
                                                                                                UBid: vidhijat  sbajoria
                                                                                            Person no:50207090  50208005

**CSE 4/587 Data Intensive Computing**
====================
***LAB 5: PROGRAMMING THE DATAFLOW FOR BIG DATA ANALYTICS USING APACHE SPARK***
---------------------------------------
                                                                                                       

*Featured Activity1: Analysis of Latin Documents for word-co-occurence: Bigrams and Trigrams*
--------------------------------------




- Importing Libraries required

In [1]:
import csv
import datetime
import string
import re

- Creating a dictionary for storing the original words with its lemmatizations
- File used: new_lemmatizer.csv

In [2]:
with open('new_lemmatizer.csv',encoding='utf8') as f: 
    reader = csv.reader(f)
    mydict = {rows[0]:rows[0:] for rows in reader}

- Creating a new SparkContext

In [3]:
sc = SparkContext.getOrCreate()

 **lemmatizeBigram**
- Creating a method to calculate the Bigrams
- Splitting each line into two parts: Location and text
- Cleaning the text, i.e. removing extra spaces and non alphanumeric characters
- Replacing v with u and j with i
- Calculating neighbours in each line and finding the corresponding lemmas for the same
- Appending the neighbour words (word1, word2) with its location to a list
- Appending the permutations of the corresponding lemmas for word1 and word2 along with its location to the same list
- Returning the list

In [4]:
def lemmatizeBigram(line) :
    fields = line.split(">", 1)
    fields[1]=fields[1].lstrip().lower().replace('v','u').replace('j','i')
    fields[1]=re.sub('[^A-Za-z0-9 ]+', '', fields[1])
    location=fields[0]+">"
    tokens=' '.join(fields[1:]).split(' ')
    lemmatizedValues=[]
    for i in range(0,(len(tokens)-1)):
        if tokens[i] in mydict:
            firstlist=mydict.get(tokens[i])
        else:
            firstlist=[tokens[i]]
        for j in range (i+1,(len(tokens))):
            if tokens[j] in mydict:
                secondlist=mydict.get(tokens[j])
            else:
                secondlist=[tokens[j]]
            for first in [x for x in firstlist if x]:
                for second in [x for x in secondlist if x] :
                    lemmatizedValues.append([','.join([first,second]),[location]])
    return lemmatizedValues


 **lemmatizeTrigram**
- Creating a method to calculate the Trigrams
- Splitting each line into two parts: Location and text
- Cleaning the text, i.e. removing extra spaces and non alphanumeric characters
- Replacing v with u and j with i
- Calculating neighbours in each line and finding the corresponding lemmas for the same
- Appending the neighbour words (word1, word2, word3) with its location to a list
- Appending the permutations of the corresponding lemmas for word1, word2 and word3 along with its location to the same list
- Returning the list

In [5]:
def lemmatizeTrigram(line) :
    fields = line.split(">", 1)
    fields[1]=fields[1].lstrip().lower().replace('v','u').replace('j','i')
    fields[1]=re.sub('[^A-Za-z0-9 ]+', '', fields[1])
    location=fields[0]+">"
    tokens=' '.join(fields[1:]).split(' ')
    lemmatizedValues=[]
    for i in range(0,(len(tokens)-2)):
        if tokens[i] in mydict:
            firstlist=mydict.get(tokens[i])
        else:
            firstlist=[tokens[i]]
        for j in range (i+1,(len(tokens)-1)):
            if tokens[j] in mydict:
                secondlist=mydict.get(tokens[j])
            else:
                secondlist=[tokens[j]]
            for k in range (j+1,(len(tokens))):
                if tokens[k] in mydict:
                    thirdlist=mydict.get(tokens[k])
                else:
                    thirdlist=[tokens[k]]
            for first in [x for x in firstlist if x]:
                for second in [x for x in secondlist if x] :
                    for third in [x for x in thirdlist if x] :
                        lemmatizedValues.append([','.join([first,second,third]),[location]])
    return lemmatizedValues


- Reading the inputs from a folder by varying the number of files
- calling the method lemmatizeBigram and applying flatmap on the obtained list
- Saving the ouotput to a output Folder Specified
- Also, calculating the time taken for the execution by noting down the start time and the end time


- Below is the execution time for 20 files

In [8]:
a = datetime.datetime.now()
lines = sc.textFile('Input\\')\
.filter(lambda x: len(x.split(">"))!=1).flatMap(lemmatizeBigram)\
.reduceByKey(lambda x,y:x+y).saveAsTextFile('OutputBigram')
b = datetime.datetime.now()
print('Time taken by bigrams',int((b - a).total_seconds()*1000),' milliseconds')

Time taken by bigrams 44599  milliseconds


- Reading the inputs from a folder by varying the number of files
- calling the method lemmatizeTrigram and applying flatmap on the obtained list
- Saving the ouotput to a output Folder Specified
- Also, calculating the time taken for the execution by noting down the start time and the end time


- Below is the execution time for 20 files

In [10]:
a = datetime.datetime.now()
lines = sc.textFile('Input\\')\
.filter(lambda x: len(x.split(">"))!=1).flatMap(lemmatizeTrigram)\
.reduceByKey(lambda x,y:x+y).saveAsTextFile('OutputTrigram')
b = datetime.datetime.now()
print('Time taken by trigrams',int((b - a).total_seconds()*1000),' milliseconds')

Time taken by trigrams 80377  milliseconds
